# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 10 2023 8:59AM,257110,12,HH-39881,Hush Hush,Released
1,Feb 10 2023 8:55AM,257115,10,0086369380,ISDIN Corporation,Released
2,Feb 10 2023 8:55AM,257115,10,0086369382,ISDIN Corporation,Released
3,Feb 10 2023 8:55AM,257115,10,0086369381,ISDIN Corporation,Released
4,Feb 10 2023 8:55AM,257115,10,0086369383,ISDIN Corporation,Released
5,Feb 10 2023 8:55AM,257115,10,0086369386,ISDIN Corporation,Released
6,Feb 10 2023 8:55AM,257115,10,0086369387,ISDIN Corporation,Released
7,Feb 10 2023 8:55AM,257115,10,0086369389,ISDIN Corporation,Released
8,Feb 10 2023 8:55AM,257115,10,0086369388,ISDIN Corporation,Released
9,Feb 10 2023 8:55AM,257115,10,0086369391,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
34,257114,Released,1
35,257115,Released,54
36,257116,Released,1
37,257117,Released,5
38,257118,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
257114,NaN,1.0
257115,NaN,54.0
257116,NaN,1.0
257117,NaN,5.0
257118,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
257004,0.0,1.0
257024,0.0,6.0
257027,0.0,1.0
257028,0.0,11.0
257030,0.0,3.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
257004,0,1
257024,0,6
257027,0,1
257028,0,11
257030,0,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,257004,0,1
1,257024,0,6
2,257027,0,1
3,257028,0,11
4,257030,0,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,257004,,1
1,257024,,6
2,257027,,1
3,257028,,11
4,257030,,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 10 2023 8:59AM,257110,12,Hush Hush
1,Feb 10 2023 8:55AM,257115,10,ISDIN Corporation
55,Feb 10 2023 8:51AM,257118,10,"Nextsource Biotechnology, LLC"
56,Feb 10 2023 8:50AM,257116,16,"SHL Pharma, LLC"
57,Feb 10 2023 8:49AM,257117,10,Emerginnova
62,Feb 10 2023 8:44AM,257114,10,ISDIN Corporation
63,Feb 10 2023 8:44AM,257111,10,ISDIN Corporation
77,Feb 10 2023 8:30AM,257094,10,SugarBear
78,Feb 10 2023 8:30AM,257089,10,ISDIN Corporation
82,Feb 10 2023 8:30AM,257098,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Feb 10 2023 8:59AM,257110,12,Hush Hush,,1
1,Feb 10 2023 8:55AM,257115,10,ISDIN Corporation,,54
2,Feb 10 2023 8:51AM,257118,10,"Nextsource Biotechnology, LLC",,1
3,Feb 10 2023 8:50AM,257116,16,"SHL Pharma, LLC",,1
4,Feb 10 2023 8:49AM,257117,10,Emerginnova,,5
5,Feb 10 2023 8:44AM,257114,10,ISDIN Corporation,,1
6,Feb 10 2023 8:44AM,257111,10,ISDIN Corporation,,14
7,Feb 10 2023 8:30AM,257094,10,SugarBear,,1
8,Feb 10 2023 8:30AM,257089,10,ISDIN Corporation,,4
9,Feb 10 2023 8:30AM,257098,10,ISDIN Corporation,,8


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 10 2023 8:59AM,257110,12,Hush Hush,1,
1,Feb 10 2023 8:55AM,257115,10,ISDIN Corporation,54,
2,Feb 10 2023 8:51AM,257118,10,"Nextsource Biotechnology, LLC",1,
3,Feb 10 2023 8:50AM,257116,16,"SHL Pharma, LLC",1,
4,Feb 10 2023 8:49AM,257117,10,Emerginnova,5,
5,Feb 10 2023 8:44AM,257114,10,ISDIN Corporation,1,
6,Feb 10 2023 8:44AM,257111,10,ISDIN Corporation,14,
7,Feb 10 2023 8:30AM,257094,10,SugarBear,1,
8,Feb 10 2023 8:30AM,257089,10,ISDIN Corporation,4,
9,Feb 10 2023 8:30AM,257098,10,ISDIN Corporation,8,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 10 2023 8:59AM,257110,12,Hush Hush,1,
1,Feb 10 2023 8:55AM,257115,10,ISDIN Corporation,54,
2,Feb 10 2023 8:51AM,257118,10,"Nextsource Biotechnology, LLC",1,
3,Feb 10 2023 8:50AM,257116,16,"SHL Pharma, LLC",1,
4,Feb 10 2023 8:49AM,257117,10,Emerginnova,5,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Feb 10 2023 8:59AM,257110,12,Hush Hush,1.0,NaN
1,Feb 10 2023 8:55AM,257115,10,ISDIN Corporation,54.0,NaN
2,Feb 10 2023 8:51AM,257118,10,"Nextsource Biotechnology, LLC",1.0,NaN
3,Feb 10 2023 8:50AM,257116,16,"SHL Pharma, LLC",1.0,NaN
4,Feb 10 2023 8:49AM,257117,10,Emerginnova,5.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 10 2023 8:59AM,257110,12,Hush Hush,1.0,0.0
1,Feb 10 2023 8:55AM,257115,10,ISDIN Corporation,54.0,0.0
2,Feb 10 2023 8:51AM,257118,10,"Nextsource Biotechnology, LLC",1.0,0.0
3,Feb 10 2023 8:50AM,257116,16,"SHL Pharma, LLC",1.0,0.0
4,Feb 10 2023 8:49AM,257117,10,Emerginnova,5.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,5655720,170.0,0.0
12,771247,3.0,0.0
15,257004,1.0,0.0
16,771316,2.0,1.0
18,257106,1.0,0.0
19,1799646,27.0,0.0
20,257027,1.0,0.0
22,257102,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,5655720,170.0,0.0
1,12,771247,3.0,0.0
2,15,257004,1.0,0.0
3,16,771316,2.0,1.0
4,18,257106,1.0,0.0
5,19,1799646,27.0,0.0
6,20,257027,1.0,0.0
7,22,257102,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,170.0,0.0
1,12,3.0,0.0
2,15,1.0,0.0
3,16,2.0,1.0
4,18,1.0,0.0
5,19,27.0,0.0
6,20,1.0,0.0
7,22,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,170.0
1,12,Released,3.0
2,15,Released,1.0
3,16,Released,2.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,22
Status,,,,,,,,
Executing,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
Released,170.0,3.0,1.0,2.0,1.0,27.0,1.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,22
0,Executing,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,Released,170.0,3.0,1.0,2.0,1.0,27.0,1.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,22
0,Executing,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,Released,170.0,3.0,1.0,2.0,1.0,27.0,1.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()